In [1]:
# Importación de módulos
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast

In [2]:
# - Rutas y comprobaciones de existencia de ficheros
from os import path
from pathlib import Path
from glob import glob
from sys import path as syspath

In [3]:
%pwd

'/home/radiofisica/Shares/Radiofisica/XiOAriaDrive'

In [4]:
# Referenciar
baseworkdir = '/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO'

Crear la carpeta `scripts` en el directorio base de trabajo en **gDrive** y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [15]:
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py


# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

# Comprobar si el plan genérico con un único campo está y si no descargarlo
if not Path('RTPlan.dcm').exists():
  !wget https://raw.githubusercontent.com/csarux/XiOrecovery/main/DICOMSets/RTPlan.dcm

--2023-08-04 11:34:52--  https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
Resolving proxy.salud.madrid.org (proxy.salud.madrid.org)... 172.22.200.10
Connecting to proxy.salud.madrid.org (proxy.salud.madrid.org)|172.22.200.10|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 7741 (7,6K) [text/plain]
Saving to: ‘/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/scripts/xioss’

xioss               100%[===================>]   7,56K  --.-KB/s    in 0,002s  

2023-08-04 11:34:53 (3,09 MB/s) - ‘/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/scripts/xioss’ saved [7741/7741]

--2023-08-04 11:34:53--  https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
Resolving proxy.salud.madrid.org (proxy.salud.madrid.org)... 172.22.200.10
Connecting to proxy.salud.madrid.org (proxy.salud.madrid.org)|172.22.200.10|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 612 [text/plain]
Saving to: ‘/home/rad

In [16]:
ArchivedPatient = '584953_SALDA_A_GARCES_SANTIAGO_Mar_14_2011.tgz'

In [17]:
%%capture
!tar -xvzf $ArchivedPatient

In [18]:
PatientID = ArchivedPatient.split('_')[0]
PatientID

'584953'

In [19]:
demographicfile = !find ./patient/$PatientID -name demographic -print
with open(demographicfile[0], 'r', encoding='latin') as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]
ApellidosNombre

'SALDAÑA GARCES,SANTIAGO'

In [20]:
!tree -d patient/$PatientID

patient/584953
├── anatomy
│   └── studyset
│       └── CT1
│           ├── DCMData
│           └── surfaces
├── dvh
└── plan
    ├── 1FASE1B
    └── PLANSETUP

9 directories


In [21]:
CT = 'CT1'
Plan ='1FASE1B'

In [32]:
%pwd

'/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient/584953/anatomy/studyset/CT1'

In [31]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
  # Copiar el script para la creación del índice de cortes de CT
  !cp scripts/genindex patient/$PatientID/anatomy/studyset/$CT

  # Cambiar el directorio de trabajo
  %cd patient/$PatientID/anatomy/studyset/$CT

  # Adjudicar permisos de ejecución y crear el índice
  !chmod +x genindex
  !./genindex

# Cambiar el directorio de trabajo
%cd patient/$PatientID/anatomy/studyset/$CT

# Crear la carpeta para los archivos máscara con las estructuras
!mkdir -p ./ss

# Copiar el script para la creación de las estructuras
!cp $baseworkdir/scripts/xioss .

# Correr el script de recuperación de las estructuras
!chmod +x xioss
!./xioss --input $baseworkdir/patient/$PatientID --plan $Plan --studyset $CT > xioss.log

/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient/584953/anatomy/studyset/CT1
[Errno 2] No such file or directory: 'patient/584953/anatomy/studyset/CT1'
/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient/584953/anatomy/studyset/CT1
mv: 'ss/patient.mha' and 'ss/patient.mha' are the same file
mv: 'ss/EXTERNO.mha' and 'ss/EXTERNO.mha' are the same file
mv: 'ss/pulmon dcho.mha' and 'ss/pulmon dcho.mha' are the same file
mv: 'ss/pulmon izqdo.mha' and 'ss/pulmon izqdo.mha' are the same file
mv: 'ss/medula.mha' and 'ss/medula.mha' are the same file
mv: 'ss/vertebra.mha' and 'ss/vertebra.mha' are the same file
mv: 'ss/esofago.mha' and 'ss/esofago.mha' are the same file
mv: 'ss/2+4+10+11 R.mha' and 'ss/2+4+10+11 R.mha' are the same file
mv: 'ss/3+5.mha' and 'ss/3+5.mha' are the same file
mv: 'ss/PTV 50.4.mha' and 'ss/PTV 50.4.mha' are the same file
mv: 'ss/corazon.mha' and 'ss/corazon.mha' are the same file
Use of uninitialized value in string eq at ./xioss line 152.
Cons

In [38]:
# Regresar al directorio base
%cd $baseworkdir

# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
convert_args_rt ={'input' :  'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/DCMData',
                  'input-prefix' : 'patient/' + PatientID  + '/anatomy/studyset/' + CT + '/ss',
                  'input-dose-xio' : 'patient/' + PatientID  + '/plan/' + Plan + '/dose.1',
                  'output-dicom' : 'xiodcm'}

# Correr el script de conversión de los datos XiO en estudio DICOM
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO

Running 'plastimatch convert' with the specified arguments:
  --input patient/584953/anatomy/studyset/CT1/DCMData
  --input-prefix patient/584953/anatomy/studyset/CT1/ss
  --input-dose-xio patient/584953/plan/1FASE1B/dose.1
  --output-dicom xiodcm
... Done.


In [39]:
# Cambiar al directorio patient
%cd $baseworkdir/patient

# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd $baseworkdir

# Incorporar el plan genérico
!cp RTPlan.dcm xiodcm

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO/patient
/home/radiofisica/Shares/Radiofisica/XiOAriaDrive/XiO
